In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from google.colab import drive
from numpy.random import seed
from keras import backend as K
import time, os, fnmatch, shutil
import pandas as pd
import numpy 
import tensorflow
tensorflow.random.set_seed(7)
seed(7)

co = 'spri'
'''
co value would iterate throught the following file names:
att_amalgam_day.csv
ver_amalgam_day.csv
spri_amalgam_day.csv
tmo_amalgam_day.csv
'''
file = r'/content/drive/My Drive/Colab Notebooks/NN/%s_amalgam_day.csv'%(co)
colName = 'Return On Sales'
'''
colName will be assigned the following values:
'Return On Assets', 
'Return On Sales', 
'Operating Income to Sales',
'Operating Income to Assets', 
'Operating Epenses to Sales', 
'Cost of Goods to Sales',
'Sellecing, General Administrative Expenses to Sales']
'''
exp = 'exp2'
metric = 'ROS'
'''
metric values will be assigned the following values:
ROS
ROA
OIA
OIS
OES
COGS
GAES
'''
years1 = ['2012','2013','2014','2015','2016','2017','2018','2019']
Q1 = '01-01'
Q2 = '04-01'
Q3 = '07-01'
Q4 = '10-01'
qtrs = [Q1,Q2,Q3,Q4]

drive.mount('/content/drive')
data = pd.read_csv(file)

#Complete the dataset 5 inputs 1 output(colName)
inputs = ['norm_score','AT&T-US', 'SPRINT-US', 'TMOBILE-US', 'VERIZON-US']
inputs.append(colName)
data = data[inputs]
print(data.shape)
train, test = train_test_split(data, test_size=0.2)
print(train.shape, test.shape)

# split into input (X) and output (Y) variables
xTrain = train.loc[:,:'VERIZON-US']
yTrain = train.loc[:,colName:]

#Custom Metric - R2
def r_squared(y_true, y_pred):
    from keras import backend as K
    SS_res =  K.sum(K.square( y_true-y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

neurons1 = [30]
neurons2 =[50]
epochs = [250]
batch_size = [40]
learn_rate = 0.001

for n1 in neurons1:
  for n2 in neurons2:
    for e in epochs:
      for b in batch_size:
        parameters = [n1,n2,e,learn_rate,b]
        cvscores = []
        
        # create model
        model = Sequential()
        model.add(Dense(n1, input_dim=5, activation='relu',kernel_initializer='normal'))
        model.add(Dense(n2, activation='relu',kernel_initializer='normal'))
        model.add(Dense(1))
        #Compile model
        optimizer = Adam(lr=learn_rate)
        model.compile(loss='mse',optimizer=optimizer,metrics=[r_squared,'mae','mse','mape'])
        #Fit the model
        history = model.fit(xTrain,yTrain, 
                epochs=e, 
                batch_size=b, 
                verbose=0)
        
        for yr in years1:
          for qtr in qtrs:
            #Iterate through the Quarterly files
            file = r'/content/drive/My Drive/Colab Notebooks/NN/Quarters/%s/%s_%s-%s.csv'%(co,co,yr,qtr)
            data1 = pd.read_csv(file)
            
            #Complete the dataset 5 inputs 1 output(colName)
            inputs = ['norm_score','AT&T-US', 'SPRINT-US', 'TMOBILE-US', 'VERIZON-US']
            inputs.append(colName)
            data1 = data1[inputs]
            print("Test Data:",data1.shape)
            xTest = data1.loc[:,:'VERIZON-US']
            yTest = data1.loc[:,colName:]
            # evaluate the model
            scores = model.evaluate(xTest,yTest, verbose=0)
            results = []
            results.append("PARAMETERS:")
            results.append(parameters)
            
            for i in range(len(model.metrics_names)):
              print("Model Name %s: %s" %(str(i),model.metrics_names[i]))
              results.append("Model Name %s: %s " %(str(i) ,str(model.metrics_names[i])))
              
              print("Model Scores %s : %s "%(str(i),str(scores[i])))
              results.append("Model Scores %s : %s " %( str(i), str(scores[i])))

            print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
            results.append("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
            cvscores.append(scores[1] * 100)
            print("\n%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores)))
            results.append("\n%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores)))
            t = time.localtime()
            timestamp = time.strftime('%b-%d-%Y_%H%M', t)
            print(parameters,timestamp)
            filename = r'/content/drive/My Drive/Colab Notebooks/RES_QTR_EXP1/%s/%s/%s_%s_%s_%s_%s.txt'%(co,metric,co,metric,exp,yr,qtr)
            with open(filename, "w") as output:
                output.write(str(results))


[Ref.1 Manning Book: Deep Learning w/Python](https://livebook.manning.com/book/deep-learning-with-python/chapter-3/297)

[Ref.2 Tutorial:NN-Python-Keras](https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/)

[Ref.3 Adam Optimization for Deep Learning](https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/)

[Ref.4 Normalize Columns-Pandas DF](https://stackoverflow.com/questions/26414913/normalize-columns-of-pandas-data-frame)

[Ref.5 GridSearch-hyperparameters-keras/](https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/)

[Ref.6 Making Predictions](https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/)

[Ref.7 Regression Prediction](https://machinelearningmastery.com/how-to-make-classification-and-regression-predictions-for-deep-learning-models-in-keras/)

[ref.8 Regresson Tutorial](https://machinelearningmastery.com/regression-tutorial-keras-deep-learning-library-python/)

[Ref.9 Metrics , Regression Tutorial](https://machinelearningmastery.com/custom-metrics-deep-learning-keras-python/)

[Ref.10 Prediction using kerasregressor](https://stackoverflow.com/questions/44132652/keras-how-to-perform-a-prediction-using-kerasregressor)

[Ref.11 Evaluating Deep Learning Models](https://machinelearningmastery.com/evaluate-performance-deep-learning-models-keras/)

[Ref.12 Reproducible Results with Keras](https://machinelearningmastery.com/reproducible-results-neural-networks-keras/
)